In this notebook we implement a simple Deep Learning dependency parser following this [paper](http://cs.stanford.edu/people/danqi/papers/emnlp2014.pdf) and the [homework](http://web.stanford.edu/class/cs224n/assignment2/index.html) from the Standford NLP course. 

# What is a dependency parser?

Given a sentence organize the words in such a way that we know which words are arguments of other words. This are called dependencies.

### Example

How should we parse the sentence "I saw an ice-cream with my phone"?

'saw'-> 'I'

'saw'-> 'ice-cream'

'saw'-> 'with'

'ice-cream' -> 'an'

'with' -> 'phone'

'phone'-> 'my'

How can we build our own parser?

## Greedy Transition-Based parsing

Let $S=w_0w_1\cdots w_n$ be a sentence, we consider the triple $$ c=(\gamma,\beta, A),$$ where

- A stach $\gamma$ of words from $S$.
- A buffer $\beta$ of words from $S$.
- A **set** of arcs $A$ of the form $(w_i,w_j)$.

The **goal** is to start from the inital state

$$c_0=\left(\gamma=['ROOT'],\beta=[w_0,w_1,\ldots,w_n],A=\emptyset\right)$$

and use the transitions 

- **Shift**: Remove first words in $\beta$ and push it on top of $\gamma$.
- **Left-Arc**: Add $(w_j,w_i)$ to $A$, where $w_i$ is the second to the top word on $\gamma$ and $w_j$ is the word at the top. Then, remove $w_i$ from $\gamma$.
- **Right-Arc**: Add $(w_i,w_j)$ to $A$, where $w_i$ is the second to the top word on $\gamma$ and $w_j$ is the word at the top. Then, remove $w_j$ from $\gamma$.

to achieve a configuration

$$c_0=\left(\gamma=['ROOT'],\beta=\emptyset,A=\right)$$

where A is now a parsing structure for the sentence.

### partialParse Object

We start by creating an object to keep the different configurations that we obtain from c, we call that object a partialParse.

In [1]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.
            We initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes.
        
        self.sentence = sentence
        self.stack=['ROOT']
        self.buffer=sentence
        self.dependencies=[]
       

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions.
        """
        
        if transition=="S":
            # Removes the first one from the buffer
            b1=self.buffer.pop(0)
            # adds it to the stack
            self.stack.append(b1)
            
        elif transition=="LA":
            #removes the second one from the stack
            s2=self.stack.pop(-2)
            
            s1=self.stack[-1]
            #adds the arc s1->s2 to the dependencies.
            self.dependencies.append((s1,s2))
            
        elif transition=="RA":
            #removes the first one from the stack
            s1=self.stack.pop(-1)
            
            s2=self.stack[-1]
            #adds the arc s2->s1 to the dependencies.
            self.dependencies.append((s2,s1))


    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies


So, for example if we have the sentence "I saw a cow", we have 

In [2]:
example_pp=PartialParse(['I','saw','a','cow'])
example_pp.parse(['S','S','LA','S','S','LA','RA'])

[('saw', 'I'), ('cow', 'a'), ('saw', 'cow')]

**Exercise:** Check the transitions that are necessary to give a correct parsing of the sentence.

# The deep learning stuff

In order to create such a parser we follow the approach of assigment 2. That is, we want to create a NN that for a given triple $c$ decides what is the next transition. 

We are given some training data, for our purposes the data looks like a collection of tables like the following:

|index|word|arrow from|
|---|---|---|
|1|Ms.|2|
|2|Haag|3|
|3|plays|0|
|4|Elianti|3|
|5|.|3|

and it will be preprocessed so it looks in the form X,y. We will go over the generalities of this during the lecture. But keep in mind that a greedy approach in the selection of the correct arcs makes the desired steps unique. 

In the homework, the method that gives the data is load_and preprocess_data inside the parser_utils module

In [3]:
from assignment2_sol.utils.parser_utils import *

In [4]:
data=load_and_preprocess_data()

Loading data...
took 3.28 seconds
Building parser...
took 0.09 seconds
Loading pretrained embeddings...
took 3.42 seconds
Vectorizing data...
took 0.10 seconds
Preprocessing training data...
1000/1000 [==============================] - 5s     


Note that we obtain 5 different objects, we only care about the second one which is the embedding matrix

In [5]:
emb_matrix=data[1]

and the third one who is the training data

In [6]:
train_data=data[2]

furthermore, the training data comes in the form of a list which each elemnt is (x,something,y), so we collect the x and the y parts

In [7]:
x_input_data=np.array([train_data_element[0] for train_data_element in train_data])
y_input_data_=np.array([train_data_element[2] for train_data_element in train_data])

We just need one more preprocessing since the y_input_data is not exactly in the shape we want

In [8]:
y_input_data = np.zeros((y_input_data_.size, 3))
y_input_data[np.arange(y_input_data_.size), y_input_data_] = 1

We are ready to build our model, first some global variables

In [9]:
import tensorflow as tf

In [10]:
n_features = 36
n_classes = 3
dropout = 0.5
embed_size = 50
hidden_size = 200
batch_size = 2048
n_epochs = 10
lr = 0.001

As always we start with the placeholders

In [11]:
input_placeholder=tf.placeholder(shape=(None,n_features),dtype=tf.int32,name='input_placeholder')
labels_placeholder=tf.placeholder(shape=(None,n_classes),dtype=tf.float32,name='labels_placeholder')
dropout_placeholder=tf.placeholder(shape=(),dtype=tf.float32,name='dropout_rate_placeholder')

we create a dictionary to feed the data to the placeholders

In [12]:
feed_dict={input_placeholder: x_input_data,
           labels_placeholder:y_input_data,
           dropout_placeholder: dropout}

and the embedding layer

In [13]:
embeddings=tf.Variable(emb_matrix)
embeddings= tf.nn.embedding_lookup(embeddings,input_placeholder)
embeddings= tf.reshape(embeddings,shape=(-1,embed_size*n_features))

and then the prediction layer

In [14]:
W=tf.Variable(tf.random_normal(shape=(n_features*embed_size,hidden_size)))
b1=tf.Variable(tf.zeros(hidden_size))
U = tf.Variable(tf.random_normal(shape=(hidden_size,n_classes)))
b2=tf.Variable(tf.zeros(n_classes))

pred = tf.nn.relu(tf.matmul(embeddings,W)+b1)

pred = tf.nn.dropout(pred,dropout)

pred = tf.matmul(pred,U)+b2


next, the loss

In [15]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=labels_placeholder))

and something to measure accuracy

In [16]:
predictions = tf.argmax(pred, 1)
correct_predictions = tf.equal(predictions, tf.argmax(labels_placeholder, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))

and the training op

In [17]:
train_op=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

We are ready to run the session.

In [18]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
for i in range(100):
    acc,loss_,_=sess.run([accuracy,loss,train_op],feed_dict=feed_dict)
    print("This is step: %d, acc=%.2f, loss=%.2f"%(i,acc,loss_),end='\r')